In [15]:
import sys
import nlputils
from nlputils import train_eval_loop
import numpy
import torch
from torch import nn
from torch.nn import functional as F

In [16]:
import nlputils.AllFunction


#corpus = ["Казнить нельзя, помиловать. Нельзя наказывать.", "Казнить, нельзя помиловать. Нельзя освободить.", "Нельзя не помиловать.", "Обязательно освободить."]
#train = ["Казнить нельзя помиловать тоже, помиловать. Нельзя наказывать.", "Казнить, казнить казнить нельзя помиловать. Нельзя освободить.", "Нельзя не помиловать. помиловать нельзя при казнить", "Обязательно казнить и освободить."]
#target = [1, 1, 0, 2]
#test_target = [0, 0, 1, 2]
train_word = [
    "Телефон просто отличный, камера снимает великолепно!",
    "Батарея быстро разряжается, ожидал лучшего.",
    "Обычный смартфон, ничего особенного.",
    "Очень шустрый, не лагает, доволен покупкой!",
    "Экран тусклый, на солнце ничего не видно.",
    "Работает без проблем, заряжается быстро.",
    "Корпус выглядит дешево, но функции нормальные.",
    "Камера мылит, ожидал лучше за такие деньги.",
    "Брал для работы, устраивает полностью.",
    "Греется при нагрузке, не очень комфортно.",
    "Удобно держать в руке, приятный дизайн.",
    "Купил маме, ей все понравилось.",
    "Разочарован качеством сборки, скрипит корпус.",
    "Работает стабильно, но звук слабоват.",
    "Камера шикарная, ночью тоже снимает отлично.",
    "Интерфейс неудобный, сложно разобраться.",
    "Дисплей яркий и насыщенный, радует глаз.",
    "Автономность слабая, приходится заряжать дважды в день.",
    "Хороший телефон за свою цену.",
    "В комплекте нет зарядки, неприятный момент.",
    "Шустро работает, ничего не тормозит.",
    "Долгожданное обновление, и оно оправдало ожидания!",
    "Экран маркий, постоянно приходится протирать.",
    "Качество динамика ужасное, звук как из бочки.",
    "Доволен покупкой, не жалею потраченных денег.",
    "Заряд держит целый день, это радует.",
    "Очень тяжелый, неудобно носить в кармане.",
    "Приятный виброотклик, чувствуется премиальность.",
    "Программное обеспечение сырое, много багов.",
    "В целом все устраивает, но могло быть лучше.",
    "Отличный смартфон, приятно пользоваться.",
    "Не хватает поддержки быстрой зарядки.",
    "Камера супер, особенно портретный режим.",
    "Экран большой, но цветопередача хромает.",
    "Лагов нет, все летает.",
    "Цена завышена, за эти деньги можно найти лучше.",
    "Очень доволен покупкой, все функции работают отлично.",
    "Громкость на максимуме слабовата, неудобно в транспорте.",
    "Заряжается молниеносно, это супер!",
    "Разочаровался, ожидал большего.",
    "Телефон стильный, но очень скользкий.",
    "Камера снимает хорошо, но иногда подтупливает.",
    "Соотношение цены и качества адекватное.",
    "Аккумулятор быстро садится, но заряжается тоже быстро.",
    "Шикарный экран, на солнце видно хорошо.",
    "Не оправдал ожиданий, батарея слабая.",
    "Легкий и удобный, приятно пользоваться.",
    "Часто зависает, это раздражает.",
    "Дизайн просто космос, очень нравится.",
    "Хорошая производительность, но автономность хромает."
]
train_labels = [
  
    2, 0, 1, 2, 0, 2, 1, 0, 2, 0, 
    2, 2, 0, 1, 2, 0, 2, 0, 1, 0, 
    2, 2, 1, 0, 2, 2, 0, 2, 0, 1, 
    2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 
    1, 1, 1, 0, 2, 0, 2, 0, 2, 1
]
train_word = [
    # Отрицательные (класс 0)
    "Качество просто отвратительное, я в жизни не видел ничего хуже. Развалилось на второй день использования, деньги на ветер.",  
    "Такое ощущение, что это говно сделали из мусора. Всё трещит, скрипит и ужасно выглядит, не берите.",  
    "Полнейший брак, ни один элемент не соответствует заявленному качеству. Производитель явно сэкономил на всём, кроме цены.",  
    "Если бы можно было поставить ноль звёзд, я бы это сделал. Развалилось почти сразу, а возврат невозможен.",  
    "Честно? Это просто насмешка над покупателями. Абсолютно бесполезная вещь, которая не работает так, как обещано.",  
    "Не ожидал, что за такие деньги можно получить настолько отвратительный дебильный товар. Всё глючит, тормозит и раздражает.",  
    "Проработало пару дней, а потом перестало включаться. Поддержка просто игнорирует мои вопросы, ужас.",  
    "Всё шатается, запах пластика отвратительный, а дерьмовая упаковка была так себе. Зря потраченные деньги.",  
    "В описании одно, а в реальности совершенно другое. Разочарован, очень сильно жалею о покупке.",  
    "Сломалось буквально через неделю. А когда обратился в магазин, мне сказали, что это моя вина.",  
    "Не соответствует фото на сайте. Вместо стильного аксессуара получил дешевую подделку.",  
    "Это худшая покупка в моей жизни. Ужасный сервис, плохое качество, деньги на ветер.",  
    "Первые несколько дней всё работало, но потом начались проблемы. Чувствую себя обманутым. Говно полное",  
    "Купил, доверившись отзывам, и пожалел. Всё сделано из дешевых материалов, использовать невозможно.",  
    "Производитель явно сэкономил на всём, кроме цены. Уже при распаковке было понятно, что это хлам.",  
    "Реально хуже некуда. Продавец заверял, что это качественный товар, но это просто мусор. Дебил только купит",  
    "Пользоваться этим невозможно. Инструкция на каком-то странном языке, и ничего не понятно.",  
    "Ощущение, что этот товар уже был в использовании. Внутри упаковки говно было что-то липкое и неприятное.",  
    "Разваливается в руках, резкий химический запах, да ещё и дефекты на корпусе. Деньги на ветер.",  
    "После недели использования могу сказать, что это просто жесть. Всё говно ломается, не работает как надо.",
    "получил утюг, он сжег мне дом ))))",
    "Заявлен как оригинал, но телефон очень быстро разряжается и нагревается",
    "Удовлетворительно,две недели ,больше щетки нет",
    "Хуже придумать сложно. Абсолютно бестолковый  для 1го класса, на вопрос что у нас над головой ребёнок отвечает небо или солнце и это вполне нормально, а не облака и деревья!!! Аналогично, что под ногами земля, а не камни...",
    "Пора изъять этот шедевр из учебной программы начальной школы. Самый ужасный и предмет в целом в начальной школе. Информация, изложенная в учебнике, не соответствует возрасту детей, предоставляется не системно, в хаотичном порядке, без закрепления материала. Особенно добило, как описана история России, начиная с...",
    "Отвратительный комплект! Море минусов, неудобств и качество фуфел! Белый кот в моих глазах упал за качество товара!",
    "складывается, невозможно подмести , не рекомендую...",
    "Швабра за 1450 руб не должна разваливаться на части",
    "Красочно Не по возрасту задания Задание с рабочей тетради не соответствует с учебником. В первом классе составьте схему дома по электричеству. Вы в своём уме?!!!!! Первоклашке!!! 3 класс Стр 13 рабочая тетрадь... С помощью карты с учебника с 18-19 привести пример, окей, открыли и где же карта?????? Иной раз взрослый человек в ступор просто входит от таких задач. В основном задания, с компьютера что то распечатать... И автор считает, что ребёнок может найти что нужно и распечатать на принтере, особенно Первоклашке. Задания для родителей, ну ни как для детей. Ребёнку не интересно тк не по возрасту задания.",
    "Все смешали, включили блендер и перевратили в жуткое варево Ребёнок пошёл учиться в школу. Значит, и родителяи пора за парту? Всем доброго дня! Выскажу и я своё мнение о программе для начальной школы.",
    "Ужасное место, я настроен на него крайне отрицательно. Магазин продает рак легких, а также мучительную жизнь. Пожалуйста, хватить настривать бизнес на человеческой зависимости от никотина.  Я конечно понимаю, что от курильщиков Россия получает огромные деньги (1,1 триллион баксов), строятся дороги, аптеки и школы, но РОССИИ ОТ ЭТОГО ПОЛЬЗЫ НЕТ. БЫСТРО УХОДИТЕ ИЗ НАШЕГО БЛАГОУСТРОЕННОГО, ЖИЛОГО, НАСТРОЕНОГО НА ПОЗИТИВ РАЙОНА!!!",
    "Большие и не открываются",
    "Огромная очередь. Пришлось есть шашлык.",
    "Россиян предупредили о новых способах мошенничества. Воришка кидает вам уведомление о необходимости уплатить налог. Жертвы послушно перечисляют свои кровные, после чего остаются без денег. В России возбуждено уголовное дело по факту мошенничества.",
    "Так, хватит постить посты с запрещёнкой!",
    "Сайт лагает бесит и всё",
    "Асу взятки",
    "Новое правило - запрещено постить контент для взрослых!",
    "ну на первой фотке эта хуйня уже 200 лет стоит",
    "Старый уже и темно там ещё",
    "Чел укусил сотрудницу Бургер Кинга. Перепутал с бургером?",
    "Законы словно сосиски: лучше не видеть, как их изготавливают",
    "Кексы раньше были лучше",
    "Одна Китаянка уже жуёт !",
    "По моему глупое название. Раньше было лучше",
    "Дорогие зрители, сегодня отмечается Мeждунapoдный дeнь лeтучиx мышeй, поэтому если вы забронировали отпуск в тропических реках, например Конго и Амазонка, то просим воздержаться, так как там пируют эти твари. И, зная Китайцев и их любовь ко всему несъедобному, ждём новости про новый вирус.",
    "Я тут учусь и мне не нравится учиться",
    "пожарная тревога оказалась фейком",
    #"Алексей Навальный является ярым критиком президента России Путина. Навальный отправляется в тюрьму на много лет. Тюрьма находится на севере России. Там очень холодно. Навальный борется с плохими вещами в России. Многие люди верят в лучшие времена. На прошлой неделе Навальный умер. Лидеры на Западе говорят, что Путин несет ответственность за смерть Навального. Сейчас, без Навального, нет другой сильной личности. Никто не борется с правительством.",
    "Светлана пойдём на скаймеку сядем нет я её сейчас убивать начну",
    "Толковый словарь русского языка Ожегов Сергей Иванович Ожегова",
    "Игра которая из самых лучших превратилась в пристанище больных дегенератов, на которую плевать своим же разработчикам.",
    "игра, ставшая помойкой для больных антисоциальных существ...",
    "Не мое. Доброго времени суток, дорогие друзья! Что сказать? Советовали мне ее очень давно, потому что люблю порой потестировать на досуге компьютерные игры. Но по итогу проиграла в нее от силы десять минут и мне стало откровенно скучно. На следующий день попробовала еще раз, но оказалось, что дело не в настроении, а просто провал и это не моя игра. Мне даже сложно представить чем таким она зацепили людей. многих затянуло, но я не нахожу для меня все, графика, суть, скучно",
    "не,тут нищие дауны",
    "игра говна , разрабы идиоты , не могут нормально отбалансить персонажей каждый патч",
    "Хз...за несколько лет игра вроде и была нормальной...но сейчас увы уже нормальной её назвать нельзя никак.",
    
    


    # Нейтральные (класс 1)
    "В целом всё неплохо, но ожидал немного большего. Работает нормально, но могло быть лучше.",  
    "За свою цену это нормальный вариант, но ничего выдающегося. Просто средний товар.",  
    "Не сказал бы, что впечатлен, но и плохим назвать не могу. Выполняет свою функцию.",  
    "Товар соответствует описанию, но без восторгов. Всё работает, но ничего особенного.",  
    "На первый взгляд всё неплохо, но при ближайшем рассмотрении есть мелкие недостатки.",  
    "Не супер, но и не провал. Обычное качество, за такие деньги — нормально.",  
    "Вроде всё работает, но есть вопросы к качеству. Надеюсь, что прослужит долго.",  
    "Не совсем то, что ожидал, но пользоваться можно. Главное, что без брака.",  
    "По качеству вопросов нет, но ожидал другого. В остальном всё соответствует описанию.",  
    "Пришло вовремя, упаковка нормальная, но есть нюансы. В целом, за такую цену неплохо.",  
    "Обычный товар, не скажу, что что-то особенное. Просто выполняет свою функцию.",  
    "Если не придираться, то можно пользоваться. Хотя есть пара минусов.",  
    "Работает, но не без недостатков. Думаю, что можно было найти вариант получше.",  
    "Нормальный вариант, но если есть возможность, лучше доплатить за что-то получше.",  
    "Не могу сказать, что в восторге, но и плохого ничего не замечено. Работает.",  
    "По качеству средне, но за такую цену — приемлемо. Надеюсь, что не подведёт.",  
    "С одной стороны, всё хорошо, но не без недостатков. В целом, можно брать.",  
    "Доставка быстрая, но сам товар немного отличается от картинки. В остальном всё нормально.",  
    "Просто товар, ничего выдающегося. Не лучше и не хуже аналогов.",  
    "За свои деньги — сойдёт. Без особых нареканий, но и без восторгов.",
    "Как по мне не очень. Сегодня произошел пожар (22.12.2020), но его благополучно потушили. Дорого, вкусно и хорошая музыка.   К сожалению этот ресторан ходят курильщики, так что будет вонять)) ",
    "Соотношение цены и качества достойное. Минус звездочку за короткий шланг (1 метр)",
    "По функционалу все нравится , но недостаток большой в том что ворс сыпется ужасно, уже 2 звена полностью выпали.",
    "В целом удобно и не удобно одновременно. Подмести ею сможете, но мне кажется, что есть варианты и получше ",
    "Хорошо подметает но не удобная",
    "Общее впечатление хорошее, но первая мысль, к чему этот набор сделали белым, ведь это для мусора Швабра идеальная, прямо хочется все ей выметать и пользоваться постоянно, а вот совок не очень устойчивый",
    "Немного сложная программа Интересная подача информации, красивые картинки и тексты. Особенно, нравится учебники за 4 класс: понятная информация по природным зонам России, хорошая подача истории. Но программа немного сложная для каждого класса, хоть и понятная",
    "нормальные педали, немного маломерят",
    "хорошие тапочки, только ноги потеют",
    "Санарчик, так я называю друга (реально) Неплохое заведение",
    "Было заперто когда я пришëл",
    "Понаоткрывали всяких. Пункты выдачи это кончено супер гуд, но этот очень даже неплохой. Другими словами нормальный другими словами хороший.",
    "И красное, и белое, и всякое короче",
    "Купил тут рыбу. Прикольно. Теперь всегда ем",
    "Как по мне не очень. Сегодня произошел пожар (22.12.2020), но его благополучно потушили. Дорого, вкусно и хорошая музыка. К сожалению этот ресторан ходят курильщики, так что будет вонять ))",
    "По словам друга, готовят нормально и даже не из котов",
    "Увидел один раз как продавали тут свинную голову. Жутко было, но мне понравилось",
    "Белый квадрат",
    "Долго ждать очередь мне показалось",
    "Не думал я конечно что игры на планшете платные ... Ну ладно",
    "Интересно светится логотип на улице. Что там внутри я не оч понял",
    "Админ не умер, если что",
    "Смотрел на чужеземном языке, мало что понял, но мне понравилось",
    "Война, но не бесконечности. У неё финал есть",
    "Финал истории людей в крутых костюмах в которых не удобно двигаться",
    "Первая часть легендарнее, но этот вызывает весёлые флешбеки",
    "Статья 20 Федерального закона № 114-ФЗ. Дети до 18 лет с трудом выезжают за границу. Они могут улететь с 2-мя родителями, с одним, если второй не против (если второй не подал документ о несогласии). Чтобы ребенок мог выезжать за границу один, у него должно быть с собой согласие на выезд (подписать которое должен хотя бы 1 родитель). Причем обратно ребенок может свободно прилететь в Россию один, ему для этого никто не сможет помешать. Если один родитель написал несогласие на выезд, то ребенок останется в России( или же ). С 18 лет никаких ограничений нет.",
    "Штуки норм, металлические",
    "Ладно, спасибо что-ли",
    "Забавное свечение здания снаружи",
    "Надеюсь, что не все подписчики умрут",
    "Те самые мальчики, когда к ним в школу должен прийти террорист",   
    "Очень странный ник. Но там кушать вкусно",
    "Это достаточно достопримечательная достопримечательность",
    "Всем доброе утро! Сейчас 18 апреля 2024 года, температура в Чебоксарах в районе +7..+13 днем. Фраза дня:",
    "Маленький, но можно заказы забирать",
    "Внимание! В связи с ростом нашего канала, просим всех участников прочитать политику пользования ( шучу, можете и не читать ... )",
    "кто грустный смайл поставил?",
    "Тут можно приобрести смешные коробочки для упаковки подарка",
    "Игрушка довольно хорошая, но меня как то не втянула. Не понимаю, как можно так много в нее играть!?",
    
    
    
    
    
    

    # Положительные (класс 2)
    "Очень доволен покупкой! Всё работает идеально, качество на высоте, рекомендую!",  
    "Это просто супер! Превзошло все мои ожидания, пользоваться одно удовольствие.",  
    "Отличный товар, который полностью оправдал свою стоимость. Работает без нареканий.",  
    "Покупкой остался очень доволен! Качество превосходное, выглядит шикарно.",  
    "Лучшее, что я покупал за последнее время. Просто нет слов, очень рад.",  
    "Качество отменное, работает идеально. Всё соответствует описанию, рекомендую!",  
    "Приятно удивлён качеством. Ожидал худшего, но оказался приятно поражён.",  
    "Давно хотел купить и вот наконец-то решился. Это стоило своих денег!",  
    "Хороший товар, оправдал все мои ожидания. Буду рекомендовать друзьям.",  
    "Превосходное качество, хорошая упаковка, быстрая доставка. Всё просто отлично!",  
    "Не пожалел ни секунды, что купил. Всё идеально, спасибо продавцу!",  
    "Очень понравилось, выглядит стильно и дорого. Работает просто замечательно.",  
    "Лучшая покупка за последнее время. Рад, что выбрал именно этот вариант.",  
    "Супер! Качество отличное, всё продумано до мелочей. Доставили быстро.",  
    "Полностью доволен покупкой. Качественный материал, приятный дизайн.",  
    "Не ожидал, что будет так круто! Просто восторг, всем рекомендую!",  
    "Смело могу сказать, что это один из лучших товаров, которые я покупал.",  
    "Очень качественная вещь, удобная и стильная. Превосходное соотношение цены и качества.",  
    "Думал, будет хуже, но оказался приятно удивлён. Очень рад покупке.",  
    "Покупкой доволен, всё супер. Если сомневаетесь — берите, не пожалеете!",
    "отличный, выиграла его в озон.всего за 1 рубль купила.спасибо озон за такую возможность.",
    "пришел в хорошем состоянии. Не активирован , все работает",
    "отличный. Оригинал. Достался считай даром. Телефоном довольна. Продавца рекомендую.",
    "отличная по цене и качеству,мощная, тихая, по сравнению с предыдущей ,всем рекомендую)",
    "Очень удобно пользоваться и хранить, качество хорошее. Спасибо производителю!",
    "Очень хорошая щетка. Совок удобный. Мало места занимает.",
    "очень нравится мужу, удобно",
    "Не знаю оригинал или нет, по штрих коду не пробивается, а так отличная обувь для работы",
    "Железная дорога идёт в сторону Казани. Хороший аэропорт",
    "Амазонской тигр. Так бы я описал свои эмоции от посещения парка. Очень понравилось.",
    "Отличное место. Жаль, что его уже больше нет. Тут было много батутов и и приятных управляющих.",
    "Вкусные блинчики с разными наполнителями",
    "Ем тут иногда ваще тема",
    "Классные цветы! Люблю говорить с цветами..",
    "Ну тут реально тема пообедать!",
    "Компания TopRussiaTeхнолоджисПерфомэнс представила новую технологию установки поручня. Данную технологию хотят внедрить во все регионы России",
    "Сдекнул оттуда свой заказ. всë прошло быстро и безболезненно.",
    "МИСТЕР БИСТ С ДНЕМ РОЖДЕНИЯ !!!",
    "Хороший фильм, хорошие спецеффекты, что-то новое для marvel",
    "Картинки хорошие, забавные",
    "Обожаю человека паука",
    "Прекрасно! Брал с огромной скидкой и жёстко этому доволен.",
    "Всë хорошо, понравилось",
    "Прекрасная игра с прекрасным сюжетом, но с долгим повествованием",
    "Отличная игра чтобы посидеть с друзьями",
    "Играю со своим не самым умным другом  вообще угар реально !!!",
    "Хороший прикол, молодцы",
    "Воу воу воу там здорово. можно купить ботинки зимние kappa поэтому и здорово",
    "Был тут. Круто тут.",
    "Писал здесь некоторую олимпиаду по физике. Всë красиво и с кайфом крч. Как будто в космос слетал.",
    "Большой выбор приколов",
    "Аптека есть и стоит на своём месте. Это круто",
    "Здесь круто и можно посидеть, погулять, поесть, покупать",
    "спасибо за отзыв, мы рады открытой критики и всегда будем учитывать ваши требования",
    "Был тут всего один раз но кормят нормально",
    "это просто ахуительно !",
    "😂 😂 🤙 умер",
    "Классные цветы! Люблю говорить с цветами ...",
    "Ну, круто че",
    "Отличный геймплей. Удовольствие от каждой завершенной игры. Качественный подбор игроков. Дружелюбное и адекватное комьюнити,которое всегда вас прикроет в разгар битвы и даст хороший совет по игре.",
    "самая интересная, загадочная, классная игра! зайдя в эту игру и поиграя в ней, можно узнать очень большую информацию о вас, и о вашу семью! всем советую поиграть в эту божественную игру",
    "Игра, топ, познакомился с парнем. Встречаемся уже год",
    "Ностальгия. Большая игра. жаль что нету войс чата. Майнкрафт это отличная игра детства, сколько воспоминаний связанных с ней, сколько потраченных часов в этой игре... В общем игра некая песочница в жанре Выживание, в которой есть как Мультиплеер так и одиночка, я лично играл в детстве больше в одиночку, строил домики, разводил животных, добывал ресурсы и тд. Игра однозначно и по сей день проявляет для меня интерес. Люблю поиграть на разных серверах с другими игроками. На серверах есть различные мини игры: Bedwars, Skywars, Eggwars, Прятки, Skyblock и тд, в общем на любой цвет и вкус. Но если вы не любите мультиплеер то можно играть хорошо и в одиночку. строить Домики, добывать ресурсы, находить пещеры, убивать мобов, ну и чтобы пройти игру это уже надо попасть в Эндер Мир и убить там дракона, в общем игра увлекательная, не жалею об большом количестве потраченном времени в ней, и рекомендую поиграть в неё всем кто давно думал играть или нет.",
    "чудесная игра, с замечательным комьюнити и превосходным балансом.",
    "Очень интересная и как по мне даже полезная",
    "Минусов не обнаружил Скачал я майнкрафт ещё в далеком 2013 году, тогда мне было 8 лет. Сейчас же могу сказать, что эта игра просто впечатляет особенно выходами своих обновлений, которые дают много возможностей обычным игрокам. Пару лет назад я особо не увлекался майнкрафтом, да и к тому же проходил майнкрафт не полностью. В этом же году весной мы с другом играли по сети и мы решились пройти эту игру полностью."
    
    
    
    
]

train_labels = [0] * 56 + [1] * 60 + [2] * 66  # Соответствующие метки

test_word = [
    "Телефон приятно держать в руке, очень удобно.",  
    "Батарея ужасная, не хватает даже на полдня.",  
    "Камера неплохая, но шумновата в темноте.",  
    "Очень быстрый, все приложения работают без тормозов.",  
    "Динамик слабый, приходится использовать наушники.",  
    "Дизайн красивый, но корпус легко царапается.",  
    "Цена адекватная, за такую стоимость хороший вариант.",  
    "Греется при зарядке, это раздражает.",  
    "Автономность отличная, заряжаю раз в два дня.",  
    "Программное обеспечение стабильное, обновления выходят регулярно."
]
test_labels = [2, 0, 1, 2, 0, 1, 1, 0, 2, 2]
assert len(train_word) == len(train_labels)
assert len(test_word) == len(test_labels)
train_tokenized = nlputils.AllFunction.tokenize_corpus(texts = train_word, minSize= 0)
test_tokenized = nlputils.AllFunction.tokenize_corpus(texts = test_word, minSize = 0)
for i in range(len(train_tokenized)):
  print(train_tokenized[i])
#vocabulary, word_doc_freq = nlputils.build_vocabulary_and_freq(tokenized_corpus)
wordid, wordfreq = nlputils.AllFunction.build_vocabulary_and_freq(train_tokenized)
UNIQUE_WORDS_N = len(wordid)
UNIQUE_LABELS_N = len(set(train_labels))
print(UNIQUE_WORDS_N, UNIQUE_LABELS_N)
#for i in range(len(DF)):
#  print(DF[i][0], end = " ")
#print()
#for i in range(len(DF)):
#  print(DF[i][1], end = " ")

['качество', 'просто', 'отвратительное', 'я', 'в', 'жизни', 'не', 'видел', 'ничего', 'хуже', 'развалилось', 'на', 'второй', 'день', 'использования', 'деньги', 'на', 'ветер']
['такое', 'ощущение', 'что', 'это', 'говно', 'сделали', 'из', 'мусора', 'всё', 'трещит', 'скрипит', 'и', 'ужасно', 'выглядит', 'не', 'берите']
['полнейший', 'брак', 'ни', 'один', 'элемент', 'не', 'соответствует', 'заявленному', 'качеству', 'производитель', 'явно', 'сэкономил', 'на', 'всём', 'кроме', 'цены']
['если', 'бы', 'можно', 'было', 'поставить', 'ноль', 'звёзд', 'я', 'бы', 'это', 'сделал', 'развалилось', 'почти', 'сразу', 'а', 'возврат', 'невозможен']
['честно', 'это', 'просто', 'насмешка', 'над', 'покупателями', 'абсолютно', 'бесполезная', 'вещь', 'которая', 'не', 'работает', 'так', 'как', 'обещано']
['не', 'ожидал', 'что', 'за', 'такие', 'деньги', 'можно', 'получить', 'настолько', 'отвратительный', 'дебильный', 'товар', 'всё', 'глючит', 'тормозит', 'и', 'раздражает']
['проработало', 'пару', 'дней', 'а', 'по

In [17]:
import scipy.sparse
import nlputils.AllFunction


VECTORIZATION_MODE = "custom_tfidf"
train_vectors = nlputils.AllFunction.vectorize_texts(train_tokenized, wordid, wordfreq, mode = VECTORIZATION_MODE, scale=True)
test_vectors = nlputils.AllFunction.vectorize_texts(test_tokenized, wordid, wordfreq, mode = VECTORIZATION_MODE, scale=True)
#dense_result = vectorize_text.toarray()
train_vectors = train_vectors.toarray()
test_vectors = test_vectors.toarray()
for row in train_vectors:
    print(" ".join(map(lambda x: f"{x:.2f}", row))) 
#print(vectorize_text)
#for text in vectorize_text:
#    print(text)
#print(wordfreq)
import scipy
train_vectors = scipy.sparse.csr_matrix(train_vectors)
test_vectors = scipy.sparse.csr_matrix(test_vectors)

[['качество', 'просто', 'отвратительное', 'я', 'в', 'жизни', 'не', 'видел', 'ничего', 'хуже', 'развалилось', 'на', 'второй', 'день', 'использования', 'деньги', 'на', 'ветер'], ['такое', 'ощущение', 'что', 'это', 'говно', 'сделали', 'из', 'мусора', 'всё', 'трещит', 'скрипит', 'и', 'ужасно', 'выглядит', 'не', 'берите'], ['полнейший', 'брак', 'ни', 'один', 'элемент', 'не', 'соответствует', 'заявленному', 'качеству', 'производитель', 'явно', 'сэкономил', 'на', 'всём', 'кроме', 'цены'], ['если', 'бы', 'можно', 'было', 'поставить', 'ноль', 'звёзд', 'я', 'бы', 'это', 'сделал', 'развалилось', 'почти', 'сразу', 'а', 'возврат', 'невозможен'], ['честно', 'это', 'просто', 'насмешка', 'над', 'покупателями', 'абсолютно', 'бесполезная', 'вещь', 'которая', 'не', 'работает', 'так', 'как', 'обещано'], ['не', 'ожидал', 'что', 'за', 'такие', 'деньги', 'можно', 'получить', 'настолько', 'отвратительный', 'дебильный', 'товар', 'всё', 'глючит', 'тормозит', 'и', 'раздражает'], ['проработало', 'пару', 'дней', '

In [18]:
test_dataset = nlputils.AllFunction.SparseFeatureDataset(test_vectors, test_labels)
train_dataset = nlputils.AllFunction.SparseFeatureDataset(train_vectors, train_labels)
#optim = torch.optim.SGD()
scheduler = lambda optim : torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor = 0.5, patience = 5, verbose = True)
model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)
print(model)
best_val_loss, best_model = train_eval_loop(model = model,
                                                     train_dataset = train_dataset,
                                                     test_dataset = test_dataset,
                                                     batch_size = 32,
                                                     criterion = F.cross_entropy,
                                                     epoch_n = 200,
                                                     lr_scheduler_ctor = scheduler,
                                                     l2_reg_alpha = 0,
                                                     lr =1e-1  
                                                     )

Linear(in_features=1283, out_features=3, bias=True)
Эпоха: 0
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  1.123025397459666
Среднее значение функции потерь на валидаци:  0.8380724390347799

Эпоха: 1
Эпоха: 5 итераций, 0.03 секунд
Среднее значение функции потерь на выборке:  0.7309852838516235
Среднее значение функции потерь на валидаци:  0.569627066453298

Эпоха: 2
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.5088997731606165
Среднее значение функции потерь на валидаци:  0.39817069967587787

Эпоха: 3
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.35630369186401367
Среднее значение функции потерь на валидаци:  0.2832764262954394

Эпоха: 4
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.25674281766017276
Среднее значение функции потерь на валидаци:  0.21108370771010718

Эпоха: 5
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.194252307

/Users/admin/Documents/Coding/Script/ProjectClassificator/my_env/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.04520832560956478
Среднее значение функции потерь на валидаци:  0.04240490930775801

Эпоха: 16
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.04196148986617724
Среднее значение функции потерь на валидаци:  0.03932939593990644

Эпоха: 17
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.038160815834999084
Среднее значение функции потерь на валидаци:  0.03689513852198919

Эпоха: 18
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.03526814033587774
Среднее значение функции потерь на валидаци:  0.0340853917102019

Эпоха: 19
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.0331894097228845
Среднее значение функции потерь на валидаци:  0.03143374901264906

Эпоха: 20
Эпоха: 5 итераций, 0.01 секунд
Среднее значение функции потерь на выборке:  0.03108880917231242
Среднее значение функции потерь на валид

In [24]:
best_model.to("cpu")
model.eval()
my_word = ["качество ужасное, теперь надо будет купить новый", "на что расчитывал продавец? наверное на то что я схаваю", "ну не скажу что это топ, просто вещь как вещь", "за свои деньги сойдет, а что вы хотели за 100 рублей", "не ожидал, очень понравилось", "телефон без дефектов, экран чистый, все ок"]
my_word = ["Иострим это наш бро"]
my_word = ["худшее мероприятие, которое я посещала! Егэ давно надо было отменить, ведь так говорил мой отец.", "Танк, гад, сьел кушетку", "Фуфло для лохов", "Задумка бомбовая, я в восторге", "пришлось купить словарь ожигова", "заболел китайским вирусом, это всё из-за ваших китайцев, которые мышей едят, к тому же летучих!", "Айфон сломаный, не приехал, экран разбит, воняет какашками", "Прикольно, но есть ряд минусов. В целом нормальная штука, пойдёт", "Кирпичи надо класть ровно!", "сейчас придет админ и забанит вас!", "Меня арестовывают, звони бате срочно", "админ к сожалению заболел", "Подстольный тенис это для чего вообще?", "Конституция России", "Конституция игры дота 2", "комьюниити дота 2", "Воняло так, что я моего престарелого соседа месяц не видел", "Мне кажется я прозрел"]
my_word = ["швабра разваливается"]
my_word = [
    "Заказал пиццу, а в коробке нашли новый закон термодинамики. Спасибо за апгрейд реальности!",
    "Ваш сервис заставил мой кактус расцвести в формате MP3. Теперь слушаю колючки в наушниках.",
    "После использования вашего продукта соседский кот начал читать мне лекции по квантовой физике. Верните старого Мурзика!",
    "Купил тут ложку — теперь мои супы спонтанно превращаются в десерты. Пришлось докупать вилку-антидот.",
    "Ваше приложение научило мою стиральную машинку философии стоицизма. Вещи возвращаются чистыми, но безразличными.",
    "Заказал коврик для мыши — получил билет в параллельную вселенную. Wi-Fi там быстрее, но законы гравитации издеваются над моими чашками.",
    "После вашего кофе у меня выросли крылья. Пришлось менять гардероб и объясняться с управдомом.",
    "Ваши кроссовки заставляют тротуарную плитку рассказывать анекдоты. Соседи думают, что я сошел с ума от смеха.",
    "Купил у вас чехол для телефона — теперь гаджет требует титуловать его 'Ваша Силиконовая Величество'.",
    "Ваш шампунь превратил мои волосы в антенну. Ловлю 16 канал спутникового ТВ, но только ток-шоу с привидениями.",
    "После вашего мастер-класса мои комнатные растения организовали профсоюз. Требуют оплачиваемый отпуск и премию за фотосинтез.",
    "Заказал скрепки — получил набор для сборки миниатюрного Большого адронного коллайдера. Бумаги скреплены, но кот теперь светится в темноте.",
    "Ваш блендер превращает овощи в мелодии. Морковь играет джаз, броккодирует как металический вокал. Супрытмично, но несъедобно.",
    "Купил у вас зеркало — оно показывает моё отражение из параллельных реальностей. Версия с хвостом и рогами явно насмехается надо мной.",
    "Ваш чайный набор заваривает не напитки, а случайные исторические события. Вчера пил 'Битву при Ватерлоо' с привкусом бергамота.",
    "После ваших курсов йоги смог свернуть пространство, но не пульт от телевизора. Парадокс.",
    "Ваши носки генерируют ритм для битбокса при ходьбе. Лифт теперь избегает меня как диджея-дилетанта.",
    "Купил у вас книжную полку — она требует пароль для доступа к классике. Шекспир оказался под замком с кодовым словом 'ToBeOrNotTo123'.",
    "Ваш зонт создает локальное лето в радиусе метра. Дождь идет только в стиле ретро-джаза.",
    "Заказал набор красок — теперь мои картины сбегают с холста. Пейзаж с озером арендовал лодку и уплыл в закат."
]
my_word = ["Всех поздравляем с 23 февраля!"]
#my_label = [0, 0]
my_word = nlputils.AllFunction.tokenize_corpus(texts = my_word, minSize= 0)
vectorized_my_word = nlputils.AllFunction.vectorize_texts(my_word, wordid, wordfreq, mode = "custom_tfidf")
#my_dataset = nlputils.AllFunction.SparseFeatureDataset(my_word)
print(type(vectorized_my_word))
print(my_word)
test_dataset = torch.tensor(vectorized_my_word.toarray(), dtype=torch.float32)
tes_dataset = test_dataset.to("cpu")
for i, text in enumerate(test_dataset):
    predicate_label = torch.argmax(best_model(test_dataset[i]), dim=-1).item()
    print(i, end = " ")
    if (predicate_label == 0):
        print("Негативно: ", end = " ")
    elif (predicate_label == 1):
        print("Нейтрально: ", end = " ")
    elif (predicate_label == 2):
        print("Позитивно: ", end = " ")
    print(my_word[i][:30])


[['всех', 'поздравляем', 'с', '23', 'февраля']]
dubag count
[[0. 0. 0. ... 0. 0. 0.]]
dubag
<class 'scipy.sparse._csr.csr_matrix'>
[['всех', 'поздравляем', 'с', '23', 'февраля']]
0 Нейтрально:  ['всех', 'поздравляем', 'с', '23', 'февраля']
